In [1]:
"""
处理分支网络的输出
输出：原图、Fake Mask等
"""

import cv2
import os
import io
import json
import numpy
import imageio
import mmcv
from skimage.measure import label


In [2]:
"""
检查文件，参数（目录，后缀）
"""

def find_file(directory, extension):
    print("[INFO] directory: " + directory)
    target_files = []
    for target_file in os.listdir(directory):
        if target_file.endswith(extension):
            print("[INFO] " + extension + "_file: " + directory + "/" + target_file)
            target_files += [target_file]
    target_files.sort()
    return target_files

In [3]:
def read_json_file(directory, json_folder_name, filename, output_directory):
    print("[INFO] read_json_file: " + directory + "/" + filename)
    prefix_filename = filename[:-5] 
    print(prefix_filename)
    jpg_name = prefix_filename + ".jpg"
    jpg_file = directory + "/" + jpg_name #原图文件
    output_jpg_name = jpg_name.replace('-','').replace('(','').replace(')','').replace('_','').replace(' ','') #输出的文件名后缀
    if os.path.exists(jpg_file):
        image = imageio.imread(jpg_file)

        with io.open(json_folder_name + "/" + filename,'r',encoding='gbk') as load_f:
            load_dict = json.load(load_f) # 配置文件
            height = image.shape[0]
            width  = image.shape[1]
            for index in range(2,15):
                roi = numpy.zeros((height,width,3)).astype('uint8')
                
                for sub_id, mark in enumerate(load_dict["shapes"]):
                    points = numpy.array(mark["points"], numpy.int32)
                    lmax = numpy.max(points[0:len(points)][:, 0])
                    lmin = numpy.min(points[0:len(points)][:, 0])
                    hmax = numpy.max(points[0:len(points)][:, 1])
                    hmin = numpy.min(points[0:len(points)][:, 1])
                    label_name = mark["label"].replace(',','')
                    # 输出原始图片
                    cv2.imwrite(output_directory + "/image/" + label_name + "_" + output_jpg_name,  mmcv.bgr2rgb(image))
                    image_gt = image.copy()
                    image_gt = cv2.rectangle(image_gt, (lmin, hmin), (lmax, hmax), color = (0,255,0), thickness = 2, lineType=cv2.LINE_AA) # 图像，点集，是否闭合，颜色，线条粗细0 255 127
                    cv2.imwrite(output_directory + "/image_gt/" + label_name + "_" + output_jpg_name,  mmcv.bgr2rgb(image_gt))

                    result_file = '/root/workspace/Thyroid_Solid_Nodule/data/preprocess/chenzhou/result/'+str(index)+'/'+ label_name + "_" + output_jpg_name
                    if os.path.exists(result_file):
                        result_image = cv2.imread(result_file)
                        # 输出原始结果
                        cv2.imwrite(output_directory + "/result/" + str(index) + '/' + label_name + "_" + output_jpg_name, mmcv.bgr2rgb(result_image))
                        result_box = result_image.copy()
                        result_box = cv2.rectangle(result_box, (lmin, hmin), (lmax, hmax), color = (0,255,0), thickness = 2, lineType=cv2.LINE_AA) # 图像，点集，是否闭合，颜色，线条粗细0 255 127
                        # 输出原始结果+GT Box
                        cv2.imwrite(output_directory + "/result_box/" + str(index) + '/' + label_name + "_" + output_jpg_name,  mmcv.bgr2rgb(result_box))
                        
                        gray_roi = numpy.zeros((height,width)).astype('uint8')
                        gray_result_image = cv2.imread(result_file, cv2.IMREAD_GRAYSCALE)
                        gray_roi[hmin:hmax,lmin:lmax]=gray_result_image[hmin:hmax,lmin:lmax]
                        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(gray_roi, connectivity=4)
                        gray_roi_bool = labels == numpy.argwhere(stats[:,-1]==numpy.max(stats[1:,-1]))[0][0]
                        for h in range(height):
                            for w in range(width):
                                if gray_roi_bool[h,w]:
                                    roi[h,w] = [255, 255, 255]
                        
#                         roi[hmin:hmax,lmin:lmax]=result_image[hmin:hmax,lmin:lmax]
                        # 输出roi图像
                        cv2.imwrite(output_directory + "/roi/" + str(index) + '/' + label_name + "_" + output_jpg_name, roi)

                        roi_gtbox = roi.copy()
                        roi_gtbox = cv2.rectangle(roi_gtbox, (lmin, hmin), (lmax, hmax), color = (0,255,0), thickness = 2, lineType=cv2.LINE_AA) # 图像，点集，是否闭合，颜色，线条粗细0 255 127
                        # 输出剪裁后结果+GT Box
                        cv2.imwrite(output_directory + "/roi_gtbox/" + str(index) + '/' + label_name + "_" + output_jpg_name,  mmcv.bgr2rgb(roi_gtbox))
                        new_lmax = 0
                        new_lmin = width
                        new_hmax = 0
                        new_hmin = height
                        for h in range(height):
                            for w in range(width):
                                if roi[h,w,0] != 0:
                                    if h > new_hmax:
                                        new_hmax = h
                                    if h < new_hmin:
                                        new_hmin = h
                                    if w > new_lmax:
                                        new_lmax = w
                                    if w < new_lmin:
                                        new_lmin = w                        
                        print(new_lmax, new_lmin, new_hmax, new_hmin)
                        roi_box=roi_gtbox.copy()
                        roi_box = cv2.rectangle(roi_box, (new_lmin, new_hmin), (new_lmax, new_hmax), color = (255, 0, 0), thickness = 2, lineType=cv2.LINE_AA) # 图像，点集，是否闭合，颜色，线条粗细0 255 127
                        # 输出剪裁后结果+GT Box+new Box
                        cv2.imwrite(output_directory + "/roi_box/" + str(index) + '/' + label_name + "_" + output_jpg_name, mmcv.bgr2rgb(roi_box))
                        roi = numpy.where(roi[:,:,] != [0,0,0], [255, 0, 0], roi)
                        mask_image = cv2.addWeighted(image, 1, roi, 1, 0, dtype=cv2.CV_32F)
                        mask_image = cv2.rectangle(mask_image, (new_lmin, new_hmin), (new_lmax, new_hmax), color = (255, 0, 0), thickness = 2, lineType=cv2.LINE_AA) # 图像，点集，是否闭合，颜色，线条粗细0 255 127
                        # 输出原图+new Box
                        cv2.imwrite(output_directory + "/mask_image/" + str(index) + '/' + label_name + "_" + output_jpg_name, mmcv.bgr2rgb(mask_image))

#                         gray_result_image = numpy.zeros((height,width)).astype('uint8')
#                         for h in range(height):
#                             for w in range(width):
#                                 if roi[h,w,0] != 0:
#                                     gray_result_image[h,w] = 255
#                         print(gray_result_image.shape)
                        
# #                         gray_result_image = cv2.imread(result_file, cv2.IMREAD_GRAYSCALE)
# #                         print(gray_result_image.shape)
#                         num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(gray_result_image, connectivity=8)
#                         print(stats[1:,-1])
#                         print(numpy.argwhere(stats[:,-1]==numpy.max(stats[1:,-1]))[0][0])
#                         mask = labels == numpy.argwhere(stats[:,-1]==numpy.max(stats[1:,-1]))[0][0]
#                         print(mask)
#                         cv2.imwrite(output_directory + "/roi/" + str(index) + '/' + label_name + "_" + output_jpg_name, mask)

#                         print(stats)


In [4]:
def create_image(jpg_file):
    image = imageio.imread(jpg_file)
    height = image.shape[0]
    width = image.shape[1]
    new_image = cv2.resize(image, (width, height), interpolation=cv2.INTER_LINEAR)
    return new_image


In [5]:
def save_jpg_file(jpg_output_file, output_image):
    cv2.imwrite(jpg_output_file, output_image)

In [6]:
# 郴州数据目录
image_folder_name = '../../data/origin/chenzhou/image/'
json_folder_name = '../../data/origin/chenzhou/json/'
# 郴州数据输出目录
output_folder_name = '../../data/preprocess/demo/result/'
# 标注图像输出目录
for index in range(0,15):
    if os.path.exists(output_folder_name + "/result_box/"+str(index)) == False:
        os.makedirs(output_folder_name + "/result_box/"+str(index))
    if os.path.exists(output_folder_name + "/mask_image/"+str(index)) == False:
        os.makedirs(output_folder_name + "/mask_image/"+str(index))
    if os.path.exists(output_folder_name + "/result/"+str(index)) == False:
        os.makedirs(output_folder_name + "/result/"+str(index))
    if os.path.exists(output_folder_name + "/image/") == False:
        os.makedirs(output_folder_name + "/image/")
    if os.path.exists(output_folder_name + "/image_gt/") == False:
        os.makedirs(output_folder_name + "/image_gt/")
    if os.path.exists(output_folder_name + "/roi_box/"+str(index)) == False:
        os.makedirs(output_folder_name + "/roi_box/"+str(index))
    if os.path.exists(output_folder_name + "/roi_gtbox/"+str(index)) == False:
        os.makedirs(output_folder_name + "/roi_gtbox/"+str(index))
    if os.path.exists(output_folder_name + "/roi/"+str(index)) == False:
        os.makedirs(output_folder_name + "/roi/"+str(index))
# json文件列表
# json_files = find_file(json_folder_name, '.json')
# json_files = ['13THY420180717154528804T.json', '39THY1420180720115201073T.json']
# json_files = ['98THY4320180820151729074.json', '107THY4620180823092511176T.json']
# json_files = ['55THY1920180723153752131T.json']
json_files = ['106THY4620180823092452358T.json']
# json_files = ['13THY420180717154528804T.json', 
#               '182-12221420180907_THY131_20180907142255572T.json', 
#               '98THY4320180820151729074.json', 
#               '55THY1920180723153752131T.json', 
#               '149-39310820180803_THY111_20180803083209135T.json', 
#               '39THY1420180720115201073T.json', 
#               '107THY4620180823092511176T.json', 
#               '197-15021020181010_THY143_20181010100227611T.json',
#               '138-04180920180801_THY1005_20180801092118768V.json', 
#               '103THY4420180822155400562T.json', 
#               '106THY4620180823092452358T.json',
#               '48THY1820180723105605185.json',
#               '191-27090920180913_THY136_20180913091148742T.json',
#              '69THY2520180725160205947T.json',
#              '131THY5620180830155731310T.json',
#              '164-35091520180815_THY120_20180815151120041T.json',
#              '16THY520180717164323160V.json',
#              '207-37380920181105_THY152_20181105093920028T.json',
#              '153-03221020180803_THY113_20180803102243041T.json',
#              '174-48191420180905_THY126_20180905142132295V.json',
#              '121THY5220180830104301820T.json',
#              '18THY620180718085559429V.json',
#              '76THY31YAJIAYAN20180730095813902T.json',
#              '193-05161420180913_THY138_20180913141631443V.json',
#              '101THY4320180820151937778T.json']

for json_file in json_files:
    read_json_file(image_folder_name, json_folder_name, json_file, output_folder_name)

# binary_map = image * 255

[INFO] read_json_file: ../../data/origin/chenzhou/image//106THY4620180823092452358T.json
106THY4620180823092452358T
492 373 401 281
492 397 359 281
492 400 401 281
492 392 401 281
488 405 359 281
487 410 363 281
492 400 388 281
492 397 388 281
492 400 383 281
492 400 389 281
492 400 387 281
492 400 387 281
492 400 388 281
